# K Nearest Neighbours (KNN)

The purpose of this notebook is to show how PCA works and how to implement it using numpy.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading Data

We will be using the [iris dataset](https://www.kaggle.com/datasets/arshid/iris-flower-dataset/data)

In [ ]:
df = pd.read_csv("../data/iris.csv")

# Split data into data we will use as our already know points (train) and points we will use to test the accuracy (test)
train = df.sample(frac=0.8)
test = df.drop(train.index)

# Split data up between features and labels
train_features = np.array(train.drop(["species"], axis=1))
train_labels = np.array(train["species"])
test_features = np.array(test.drop(["species"], axis=1))
test_labels = np.array(test["species"])

# Scaling data
train_features = (train_features - np.mean(train_features, axis=0)) / np.std(train_features, axis=0)
test_features = (test_features - np.mean(test_features, axis=0)) / np.std(test_features, axis=0)

## Prediction using KNN

To predict the class of a data point we will:
1. Calculate the distances to every classified point in our dataset (we will be using the euclidian distance)
2. Find the k points with the smallest distance from our data point
3. Find the modal class of the closest data poins

In [ ]:
def knn_on_sample(data: tuple[np.array, np.array], sample: np.array, k):
    data_features = data[0]
    
    # Calculating the euclidian distance to every point
    distances = np.linalg.norm(data_features - sample, axis=1)
    
    # Find the k closest points
    sorted_indices = np.argsort(distances)[:k]
    neighbours = data[1][sorted_indices]
    
    # Return the modal class of the neighbours
    labels, counts = np.unique(neighbours, return_counts=True)
    return labels[counts.argmax()]

In [ ]:
def knn_on_samples(data: tuple[np.array, np.array], samples: np.array, k):
    prediction = np.array([knn_on_sample(data, features, k) for features in samples])
    return prediction

## Test the Accuracy of the model

Using our testing data we will calculate the accuracy of our model

In [ ]:
predictions = knn_on_samples((train_features, train_labels), test_features, 4)

# Calculate percentage accuracy
accuracy = np.count_nonzero(predictions == test_labels) / test_labels.shape[0] * 100

print(f"Accuracy = {accuracy:.2f}%")

## Visulising classification

For use to visualize the data on a 2D graph, we apply PCA to reduce the dimenstionality of the data.

For the data points in the test data set we plot them on the graph and then color them green if they were correctly classified and red if they are not.

*The code below produces this graph*

![KNN Output](../Images/KNNOutput.png)

From the graph we can see the point in the purple region have all been classified correctly. The points which have not been label correctly are near the boundary of the blue and yellow data points. When using KNN, points are likely to be miss classified if they are near the border of two different classes.

In [ ]:
import sys
sys.path.append("../")
from PCA.pca import calculate_principal_components, project_onto_components

species_map = {
    "Iris-setosa": 0,
    "Iris-versicolor": 1,
    "Iris-virginica": 2,
}

outcomes = np.array([species_map[x] for x in train_labels])

# Calculate principla components and then project data onto principal components
components, _ = calculate_principal_components(train_features)
train_projections = project_onto_components(train_features, components=components)
test_projections = project_onto_components(test_features, components=components)

plt.xlabel(f"Principal Component 1")
plt.ylabel(f"Principal Component 2")
plt.scatter(train_projections[:, 0], train_projections[:, 1], c=outcomes, s=3)

# Colors point based on if they were correctly classified
test_colors = np.vectorize(lambda x: "#00FF00" if x else "#FF0000")(predictions == test_labels)
plt.scatter(test_projections[:, 0], test_projections[:, 1], c=test_colors, s=15)

plt.show()